In [ ]:
import pandas as pd

# Load data from csv files
df = pd.read_csv(
    "2022-2023 Football Player Stats.csv", encoding="ISO-8859-1", delimiter=";"
)

# Print the first 5 rows
df.head()

In [ ]:
# Remove rows with column 'Comp' != 'Premier League'
df = df[df["Comp"] == "Premier League"]

In [ ]:
# Load point values from csv file

df_points = pd.read_csv("players.csv")
df_points = df_points[["name", "total_points", 'expected_goals', 'expected_assists']]

In [ ]:
from fuzzywuzzy import process

def get_best_match(name, choices, threshold=60):
    match = process.extractOne(name, choices, score_cutoff=threshold)
    return match[0] if match else None

In [ ]:
df["Player"] = df["Player"].apply(lambda x: get_best_match(x, df_points["name"]))

# Create a dataset with the Player and Matched Name columns to review manually in dataviewer
df2 = df[["Player", "matched_name"]]

# Drop rows with no match
df = df.dropna(subset=["Player"])

In [ ]:
# Merge the two datasets on the matched_name column 
df = df.merge(df_points, how="left", left_on="Player", right_on="name")
df = df.drop(columns=["name"])

In [ ]:
# Remove players with minutes played < threshold
df = df[df["Min"] > 90]

In [ ]:
# Drop columns that are not needed
df = df.drop(
    columns=["Rk", "Nation", "Squad", "Comp", "Age", "Born", "MP", "Starts", "90s", 'Tkl+Int']
)

# Drop columns that have direct influence on points
df = df.drop(
    columns=[
        "Goals",
        "G/Sh",
        "G/SoT",
        "ShoPK",
        "Assists",
        "GcaPassLive",
        "GcaPassDead",
        "GcaDrib",
        "GcaSh",
        "GcaFld",
        "GcaDef",
        "CrdY",
        "CrdR",
        "2CrdY",
        "OG",
        "PKwon",
        "PKcon",
    ]
)

In [ ]:
# Calculate quantiles for points and add column to dataframe
df["points_quantile"] = pd.qcut(
    df["total_points"], 4, labels=False
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Checking Pearsons correlation with total_points

numeric_df = df.select_dtypes(exclude="object")
correlation_with_points = numeric_df.corr()["total_points"].sort_values(ascending=False)
correlation_with_points = correlation_with_points.drop(['points_quantile'])

# Plot top 3 correlations vs points using scatterplot
for col in correlation_with_points.index[1:4]:
    sns.scatterplot(x=col, y="total_points", data=df)
    plt.show()

# Plotting top 10 correlations with total_points
# sns.heatmap(numeric_df[correlation_with_points.index[:10]].corr(), annot=True)